# Calculate Phi using pO2_bar, T_bar

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_{tres}.nc'
ds1 = xr.open_dataset(filename)
T_mean = ds1['TEMP_mean']
##
filename = f'{path}/DPLE-FOSI_hindcast_pO2_mean_dimension_reduced_1948-2017_{tres}.nc'
ds1 = xr.open_dataset(filename)
pO2_mean = ds1['pO2_mean']

### Calculate Φ using 13 different Eo values and the same Ac value

In [4]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [5]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'Phi_{nrow*i+j}'] = mi.Phi(pO2_mean, T_mean, Ac, Eo, dEodT=0.022)
dss

10 -0.2
10 -0.1
10 0.0
10 0.10000000000000003
10 0.2
10 0.3
10 0.4000000000000001
10 0.5
10 0.6000000000000001
10 0.7
10 0.8
10 0.9000000000000001
10 1.0000000000000002
CPU times: user 208 ms, sys: 26.1 ms, total: 234 ms
Wall time: 250 ms


<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    Phi_0    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_1    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_2    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_3    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_4    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_5    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    ...       ...
    Phi_7    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_8    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_9    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_10   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_11   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_12   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
Dimensions without coordinates: nlat, nlon
Data variables: (12/13)
    Phi_0    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_1    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_2    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_3    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_4    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_5    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    ...       ...
    Phi_7    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_8    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_9    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_10   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_11   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    Phi_12   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [7]:
%%time
dss.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_1948-2017_{tres}.nc', mode='w')

CPU times: user 31.1 ms, sys: 22.6 ms, total: 53.7 ms
Wall time: 134 ms
